<a href="https://colab.research.google.com/github/andrewdk1123/KoSentiment/blob/main/KoGPTDataAugmentor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load packages

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import string
import random
from transformers import AutoTokenizer, TFGPT2LMHeadModel, BertTokenizer
from nltk.tokenize import sent_tokenize

time: 741 µs (started: 2023-11-30 01:23:00 +00:00)


In [ ]:
import nltk

nltk.download('punkt')  # Download the punkt tokenizer models

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

time: 67.5 ms (started: 2023-11-30 00:45:34 +00:00)


# Load data to be augmented

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving pos_training.csv to pos_training (1).csv
time: 40 s (started: 2023-11-30 00:45:38 +00:00)


In [ ]:
print(uploaded.keys())

dict_keys(['pos_training (1).csv'])
time: 425 µs (started: 2023-11-30 00:46:22 +00:00)


In [ ]:
import pandas as pd
import io

pos_training_data = pd.read_csv(io.BytesIO(uploaded['pos_training (1).csv']))
pos_training_data.head()

,Unnamed: 0,emotion,sentence,label,tokenized_sentence,cleaned_tokens
0,7,E66,퇴사한 지 얼마 안 됐지만 천천히 직장을 구해보려고.,1,"['퇴', '##사한', '지', '얼마', '안', '됐', '##지만', '천천...","['퇴', '지', '얼마', '안', '됐', '천천히', '직장', '구해']"
1,9,E66,요즘 직장생활이 너무 편하고 좋은 것 같아!,1,"['요즘', '직장', '##생활', '##이', '너무', '편하고', '좋은',...","['요즘', '직장', '너무', '편하고', '좋은', '것', '같아']"
2,10,E66,취업해야 할 나이인데 취업하고 싶지가 않아.,1,"['취업', '##해야', '할', '나이', '##인데', '취업', '##하고'...","['취업', '할', '나이', '취업', '싶지', '않아']"
3,22,E64,우리 회사는 정말 사내 분위기가 좋아.,1,"['우리', '회사는', '정말', '사내', '분위기가', '좋아', '.']","['우리', '회사는', '정말', '사내', '분위기가', '좋아']"
4,23,E64,오늘 내가 다니는 회사가 참 좋은 직장이라는 생각이 들었어.,1,"['오늘', '내가', '다니는', '회사가', '참', '좋은', '직장', '#...","['오늘', '내가', '다니는', '회사가', '참', '좋은', '직장', '생..."


time: 71.8 ms (started: 2023-11-30 00:46:36 +00:00)


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Nov 30 00:46:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    42W / 300W |   1690MiB / 16384MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Augment sentences

KoGPT-2의 Language Model을 이용해서 긍정적인 문장을 생성합니다. 다음의 두 가지 가정으로부터 생성된 문장들이 긍정적인 감정을 나타낸다고 판단하였습니다.

1. 텍스트 문장에 포함된 각 단어는 긍정적인지 부정적인지의 감정을 전달한다.
2. 긍정적인 감정을 나타내는 단어들은 긍정적인 감정의 다른 단어들과 함께 나타날 가능성이 높다.

생성하는 시퀀스의 최대 길이는 50으로 하였고, Seed Sentence를 입력 하였을 때, 가장 가능성이 높은 1가지 (K=1) 토큰으로부터 문장을 만들었습니다.

In [ ]:
def generate_sentence_with_top_k(seed_sentence, model, tokenizer, max_length=50, top_k=1):
    input_ids = tokenizer.encode(seed_sentence)

    while len(input_ids) < max_length:
        output = model(tf.convert_to_tensor([input_ids]))

        # Get the top k indices
        top_k_indices = tf.math.top_k(output.logits[0, -1], k=top_k).indices.numpy()

        # Randomly select the next token from the top k
        selected_token_id = random.choice(top_k_indices)
        input_ids.append(selected_token_id)

    generated_sentence = tokenizer.decode(input_ids)
    return generated_sentence

def main(pos_training):
    model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
    tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

    generated_sentences = []

    for index, row in pos_training.iterrows():
        seed_sentence = row['sentence']
        generated_sentence = generate_sentence_with_top_k(seed_sentence, model, tokenizer)
        generated_sentence = generated_sentence.replace(seed_sentence, '')
        generated_sentences.append({
            'emotion': row['emotion'],
            'generated_sentence': generated_sentence
        })
        print(f"Seed Sentence: {seed_sentence}\nGenerated Sentence: {generated_sentence}\n")

    generated_df = pd.DataFrame(generated_sentences)
    print(generated_df)

time: 1.03 ms (started: 2023-11-30 01:32:23 +00:00)


In [ ]:
main(pos_training_data)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'lm_head.weight']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

Streaming output truncated to the last 5000 lines.
"그럼, 그게 무슨

Seed Sentence: 오늘 괴롭힌 무리들한테 통쾌하게 복수했어. 완전 신나.
Generated Sentence:  그게 다 내 잘못이야. 그게 다 내 잘못이야. 그게 다 내 잘못이야. 그게 다 내 잘못이야. 그게 다 내 잘못이야

Seed Sentence: 내가 원하는 대학에 갈 수 있을 것 같아.
Generated Sentence:  그게 바로 내가 원하는 대학입학 전형이니까. 그게 바로 내가 원하는 대학입학 전형이니까. 그게 바로 내가 원하는 대학입학 전형이니까. 그게 바로 내가 원하는

Seed Sentence: 결혼한 딸이 이번에 용돈을 올려줘서 고마워.
Generated Sentence:  내가 너를 위해 돈을 많이 벌게 해줘서 고마워. 내가 너를 위해 돈을 많이 벌게 해줘서 고마워. 내가 너를 위해 돈을 많이 벌

Seed Sentence: 돈은 역시 비서에게 맡겨야 해.
Generated Sentence:  그게 무슨 말이냐?"
"그럼, 그게 무슨 말이냐?"
"그럼, 그게 무슨 말이냐?"
"그럼, 그게 무슨 말이냐?"
"

Seed Sentence: 딸 결혼 준비로 사둔 오피스텔 가격이 많이 올라서 기뻐!
Generated Sentence: !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

Seed Sentence: 은퇴하고 집 하나 달랑 있는데 보유세가 왜 이렇게 많이 나와.
Generated Sentence:  그게 무슨 세금이냐. 그게 무슨 세금이냐. 그게 무슨 세금이냐. 그게 무슨 세금이냐. 그게 무슨 세금이냐. 그

Seed Sentence: 친구가 한동안 연락이 끊겼지만 언젠가 연락을 줄거라고 믿어.
Generated Sentence:  그게 바로 내가 너를 사랑한 이유야. 그게 바로 너를 사랑한 이유야. 그게 바로 너를 사랑한 이유야. 그게 바로 너

Seed Sentence: 대인 관계에 흥분돼.
Ge

# Export dataframe as csv file from google colab

In [ ]:
augmented_df.to_csv('augmented_df.csv')

NameError: ignored

CSV 파일을 다운로드 받습니다.

In [ ]:
from google.colab import files
files.download('augmented_df.csv')